In [12]:
#
# avatar service: changes robot controller registers dedicated to the cutting tool and end effector
#                 as per sequencer request so that action of the operator is simulated 
#

#!/usr/bin/env python
import pika, sys, time, os, json, yaml

cde_request = lambda body, reg, desc: {
                        "action": "REQUEST",
                        "definition": {
                            "api": "/stringRegister/single",
                            "body": body,
                            "method": "PUT",
                            "query": {
                                "reg": reg,
                                "type": "int"
                                }
                            },
                        "description": desc,
                        "target": "PROXY"
                        }


# list of the possible manipulation
manipulation_list = ["LOAD", "UNLOAD"]

# possible values for register 150 and 151
with open("MARS_avatar.yaml", 'r') as stream:
    eqt_list = yaml.safe_load(stream)

# waiting time prior register update 
TEMPO = 10

def callback(ch, method, properties, msg):
    body=json.loads(msg)
    if body["manipulation"] in manipulation_list:
        desc = "{} {} {}".format(body["manipulation"], body["equipment"]["type"], body["equipment"]["reference"])
        print(" [x] {} in progress".format(desc))
        time.sleep(TEMPO)
        ch.basic_publish(exchange='mars', routing_key="hmi.sequencer.report", body="OK")
        reg=eqt_list[body["equipment"]["type"]]["registerID"]
        if body["manipulation"] == "UNLOAD":
            body["equipment"]["reference"] = "NONE"
        val = eqt_list[body["equipment"]["type"]]["value"][body["equipment"]["reference"]]
        ch.basic_publish(exchange='mars', routing_key="hmi.proxy.request", 
                         body=json.dumps(cde_request(val, reg, desc)))
        print(" [x] Sent to proxy service : %s" % json.dumps(cde_request(val,reg,desc)))
    else:
        print(" [x] unknown action")
        ch.basic_publish(exchange='mars', routing_key="hmi.sequencer.report", body="NOK")
    ch.stop_consuming()

def main():
        
    connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))
    channel = connection.channel()
    channel.exchange_declare(exchange='mars', exchange_type='topic')

    result = channel.queue_declare('', exclusive=True)
    queue_name = result.method.queue
    channel.queue_bind(exchange='mars', queue=queue_name, routing_key="sequencer.hmi.request")

    print(' [*] Waiting for request from sequencer. To exit press CTRL+C')    
    channel.basic_consume(queue=queue_name, on_message_callback=callback, auto_ack=True)
    channel.start_consuming()
    connection.close()
    print(" [*] connection closed")

if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt:
        print('Interrupted')
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)

{'EFFECTOR': {'registerID': 150, 'value': {'NONE': 1, 'WEB_C_DRILLING': 2, 'FLANGE_C_DRILLING': 3, 'WEB_C_FASTENING_RIVET': 4, 'WEB_C_FASTENING_LGP': 5, 'FLANGE_C_FASTENING_LGP': 6}}, 'CUTTING TOOL': {'registerID': 151, 'value': {'NONE': 1, 'E53233043200': 2, 'ASNA2392-3-04': 3, 'ASNA2392-3-05': 4, 'ASNA2392-3-07': 5}}}
 [*] Waiting for request from sequencer. To exit press CTRL+C
 [x] LOAD EFFECTOR WEB_C_DRILLING in progress
 [x] Sent to proxy service : {"action": "REQUEST", "definition": {"api": "/stringRegister/single", "body": 2, "method": "PUT", "query": {"reg": 150, "type": "int"}}, "description": "LOAD EFFECTOR WEB_C_DRILLING", "target": "PROXY"}
 [*] connection closed
